<h3>Transfer Learning for Unsupervised Anomaly Detection</h3>

![MVTEC Image](https://www.mvtec.com/fileadmin/_processed_/1/e/csm_dataset_overview_large_6f330dede4.png)
>Image Source: https://www.mvtec.com/

MVTec AD is an open access dataset (see [MVTec dataset](https://www.mvtec.com/company/research/datasets/mvtec-ad)) used for training anomaly detection ML algorithms. It contains over 5.000 high-res images divided into 15 different objects, some of which are characterised by a morphological anomaly. These anomalies are manifested in a myriad of forms such as scratches, dents, or other structural irregularities.

<h3>The goal of this project</h3>
        
- In this IPython notebook we will use Transfer Learning, a popular approach in deep learning, to firstly distinguish objects in pristine condition from those with anomalies and secondly to detect the exact location of any potential physical defects. Transfer Learning, in case you are unfamiliar with the concept, is a technique used to expediate the training period of deep convolutional neural networks. This is achieved by utilizing weights from pre-trained models that were developed for visual object recognition software research. Top performing models can be downloaded and used directly, or integrated into a new model to solve new tasks.

<h4>Step 1 - Import Modules and data subsets</h4>

In [ ]:
# Imprort standard modules
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

# Import custom modules
from python_utils.dataloader import get_train_test_loaders, get_cv_train_test_loaders
from python_utils.model import CustomVGG
from python_utils.helper import train, evaluate, predict_localize
from python_utils.constants import NEG_CLASS

# Change the current working directory to desired path.
data_folder = os.getcwd()

# Leave as is if you wish to analyze the 'carper' subset, or change it to 'capsule' if you want to analyze the capsule dataset
subset_name = "carpet"

data_folder = os.path.join(data_folder, subset_name)

<h4>Step 2 - Define the parameters of the model </h4>

In [4]:
batch_size = 10
target_train_accuracy = 0.98
lr = 0.0001
epochs = 10
class_weight = [1, 3] 

heatmap_thres = 0.7
n_cv_folds = 5

# Move the device to "GPU" -if one is available- to expediate the analysis
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

<h4>Step 3 - Run the dataloaders </h4>

In [ ]:
# This custom function will splits dataset in stratified manner, considering various defect types.

train_loader, test_loader = get_train_test_loaders(
    root=data_folder, batch_size=batch_size, test_size=0.2, random_state=42,
)

<h4>Step 4 - Train the model </h4>

The custom-made model developed for this project is based on the VGG-16 architecture which had been pre-trained on ImageNet. The intermediary fully-connected neural network layers are replaced by an average global pooling layer and the last dense layer outputs a two-dimensional vector which is normalized and transformed into a probability distribution consisting of K probabilities ('Anomaly' for objects with structural irregularities or 'Good' otherwise). The loss-function for this model is based on cross-entropy and the optimizer is Adam with a learning rate of 0.0001. 

![Model Architrecture](https://user-images.githubusercontent.com/71797206/169629507-91206559-c316-40da-a129-d21b44931154.png)

In [ ]:
model = CustomVGG()

class_weight = torch.tensor(class_weight).type(torch.FloatTensor).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weight)
optimizer = optim.Adam(model.parameters(), lr=lr)
model = train(
    train_loader, model, optimizer, criterion, epochs, device, target_train_accuracy
)

<h4>Step 5 - Perform cross-validationa and Evaluate the model </h4>

In [ ]:
evaluate(model, test_loader, device)

## Cross Validation
cv_folds = get_cv_train_test_loaders(
    root=data_folder,
    batch_size=batch_size,
    n_folds=n_cv_folds,
)

class_weight = torch.tensor(class_weight).type(torch.FloatTensor).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weight)

for i, (train_loader, test_loader) in enumerate(cv_folds):
    print(f"Fold {i+1}/{n_cv_folds}")
    model = CustomVGG()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model = train(train_loader, model, optimizer, criterion, epochs, device)
    evaluate(model, test_loader, device)

<h4>Step 6 - Visualize the results </h4>

In [ ]:
# Runs predictions for the samples in the dataloader and shows an image with its true label, predicted label and its probability.

predict_localize(
    model, test_loader, device, thres=heatmap_thres, n_samples=3, show_heatmap=False
)

<h4> Final remarks</h4>

The model was trained on two subsets of the MVTec Anomaly Detection Dataset, namely on the 'carpet' and 'capsule' sets. As depicted in the table below, the balanced accuracy of the model on an unseen dataset was close to 90% for both the carpet and capsule subsets, which is considered acceptable for the explorative nature of this project.

Table 1: Summary of the findings 
    
Subset Title | Test Accuracy | Balanced Test Accuracy | Confusion Matrix Labels |
--- | --- | --- | --- |
Carpet | 0.91 | 0.88 | TP = 57, TN = 15, FP = 3, FN = 4 |
Capsule  | 0.93 | 0.91| TP = 46, TN = 19, FP = 3, FN = 2 |

A simple illustration of the anomalies detected.

![Findings](https://user-images.githubusercontent.com/71797206/169630481-6b613330-e16c-48b5-bb64-0db4e0dbee20.png)